### 데이터 확인

1. DB 접속

In [2]:
import pandas as pd
from pymongo import MongoClient

# === [설정] ===
MONGO_URI = "mongodb://localhost:27017/"
DB_NAME = "f1_analytics_db"

# DB 연결
client = MongoClient(MONGO_URI)
db = client[DB_NAME]

print(f"✅ MongoDB Connected: {DB_NAME}")

✅ MongoDB Connected: f1_analytics_db


2. 전체 데이터 현황

In [3]:
# 컬렉션 목록 가져오기
collections = db.list_collection_names()
summary_data = []

print(f"📊 전체 컬렉션 개수: {len(collections)}개")

for col_name in collections:
    # 문서 개수 카운트
    count = db[col_name].count_documents({})

    # 용량 확인 (MB 단위)
    stats = db.command("collstats", col_name)
    size_mb = stats['size'] / (1024 * 1024)

    summary_data.append({
        "Collection": col_name,
        "Documents": count,
        "Size (MB)": round(size_mb, 2)
    })

# DataFrame으로 깔끔하게 출력
df_summary = pd.DataFrame(summary_data).sort_values("Documents", ascending=False)
display(df_summary)

📊 전체 컬렉션 개수: 4개


,Collection,Documents,Size (MB)
2,telemetry,215925506,47542.93
3,laps,220157,47.90
0,results,6837,1.64
1,races_meta,342,0.08


3. races_meta(경기 정보) 샘플 확인

In [7]:
print("🔎 [races_meta] 샘플 데이터 (상위 5개)")

# 5개만 가져오기
cursor = db.races_meta.find().limit(5)
df_meta = pd.DataFrame(list(cursor))

if not df_meta.empty:
    # _id 컬럼은 길어서 보기 불편하므로 제거하고 출력
    display(df_meta.drop(columns=['_id'], errors='ignore'))
else:
    print("❌ 데이터가 없습니다.")

🔎 [races_meta] 샘플 데이터 (상위 5개)


,Avg_AirTemp,Avg_TrackTemp,Circuit,Country,Date,IsRainy,Location,Round,SessionType,TotalLaps,Year
0,24.7,30.5,Australian Grand Prix,Australia,2018-03-24 06:00:00,False,Melbourne,1,Qualifying,NaN,2018
1,24.1,36.3,Australian Grand Prix,Australia,2018-03-25 05:10:00,True,Melbourne,1,Race,58.0,2018
2,28.2,31.9,Bahrain Grand Prix,Bahrain,2018-04-07 15:00:00,False,Sakhir,2,Qualifying,NaN,2018
3,28.0,32.2,Bahrain Grand Prix,Bahrain,2018-04-08 15:10:00,False,Sakhir,2,Race,57.0,2018
4,12.8,15.5,Chinese Grand Prix,China,2018-04-14 06:00:00,False,Shanghai,3,Qualifying,NaN,2018


4. results(경기 결과) 샘플 확인

In [6]:
print("🔎 [results] 샘플 데이터 (상위 5개)")

cursor = db.results.find().limit(5)
df_results = pd.DataFrame(list(cursor))

if not df_results.empty:
    display(df_results.drop(columns=['_id'], errors='ignore'))
else:
    print("❌ 데이터가 없습니다.")

🔎 [results] 샘플 데이터 (상위 5개)


,Year,Round,SessionType,Driver,DriverNumber,Team,Position,GridPosition,Points,Status,Q1,Q2,Q3,TimeDelta
0,2018,1,Qualifying,HAM,44,Mercedes,1,0,0.0,,0 days 00:01:22.824000,0 days 00:01:22.051000,0 days 00:01:21.164000,None
1,2018,1,Qualifying,RAI,7,Ferrari,2,0,0.0,,0 days 00:01:23.096000,0 days 00:01:22.507000,0 days 00:01:21.828000,None
2,2018,1,Qualifying,VET,5,Ferrari,3,0,0.0,,0 days 00:01:23.348000,0 days 00:01:21.944000,0 days 00:01:21.838000,None
3,2018,1,Qualifying,VER,33,Red Bull Racing,4,0,0.0,,0 days 00:01:23.483000,0 days 00:01:22.416000,0 days 00:01:21.879000,None
4,2018,1,Qualifying,RIC,3,Red Bull Racing,5,0,0.0,,0 days 00:01:23.494000,0 days 00:01:22.897000,0 days 00:01:22.152000,None


5. laps(랩 타임) 샘플 확인

In [8]:
print("🔎 [laps] 샘플 데이터 (상위 5개)")

cursor = db.laps.find().limit(5)
df_laps = pd.DataFrame(list(cursor))

if not df_laps.empty:
    # 주요 컬럼만 보기
    cols = ['Year', 'Round', 'Driver', 'LapNumber', 'LapTime_Sec', 'Compound', 'TyreLife']
    display(df_laps[cols].head())
else:
    print("❌ 데이터가 없습니다.")

🔎 [laps] 샘플 데이터 (상위 5개)


,Year,Round,Driver,LapNumber,LapTime_Sec,Compound,TyreLife
0,2018,1,HAM,1,107.929,ULTRASOFT,1
1,2018,1,HAM,2,83.802,ULTRASOFT,2
2,2018,1,HAM,3,112.536,ULTRASOFT,3
3,2018,1,HAM,4,83.263,ULTRASOFT,4
4,2018,1,HAM,5,118.751,ULTRASOFT,5


6. telemetry(초당 주행 데이터) 상세 점검

In [9]:
print("🔎 [telemetry] 샘플 데이터 및 구조 점검")

cursor = db.telemetry.find().limit(5)
df_tel = pd.DataFrame(list(cursor))

if not df_tel.empty:
    # 1. 데이터 샘플 출력
    display(df_tel.drop(columns=['_id'], errors='ignore'))

    # 2. 드라이버 ID 타입 체크
    sample_driver = df_tel['Driver'].iloc[0]
    print(f"\n👉 [중요] Driver 컬럼 데이터 예시: {sample_driver}")
    print(f"👉 [중요] Driver 컬럼 데이터 타입: {type(sample_driver)}")

    # 3. LapNumber 컬럼 존재 여부 체크
    if 'LapNumber' in df_tel.columns:
        print("✅ LapNumber 컬럼이 존재합니다. (분석이 쉽습니다)")
    else:
        print("⚠️ LapNumber 컬럼이 없습니다. (거리 기반 슬라이싱 필요)")
else:
    print("❌ 텔레메트리 데이터가 없습니다.")

🔎 [telemetry] 샘플 데이터 및 구조 점검


,Date,SessionTime,Distance,Speed,RPM,Throttle,Brake,DRS,X,Y,Z,Year,Round,SessionType,Driver
0,2018-03-24 06:03:06.303,426.138,0.532643,111.287507,10684.142494,60.683337,False,8,-10222.000028,-6819.000026,87.0,2018,1,Qualifying,44
1,2018-03-24 06:03:06.322,426.157,1.143611,112.000000,10766.000000,61.000000,False,8,-10222.000028,-6819.000026,87.0,2018,1,Qualifying,44
2,2018-03-24 06:03:06.465,426.300,5.850070,116.766669,10020.016333,67.554170,False,8,-10222.000000,-6819.000000,87.0,2018,1,Qualifying,44
3,2018-03-24 06:03:06.562,426.397,9.143611,120.000000,9514.000000,72.000000,False,8,-10221.999951,-6818.999955,87.0,2018,1,Qualifying,44
4,2018-03-24 06:03:06.765,426.600,16.310646,126.766669,9984.283482,67.770832,False,8,-10222.000000,-6819.000000,87.0,2018,1,Qualifying,44



👉 [중요] Driver 컬럼 데이터 예시: 44
👉 [중요] Driver 컬럼 데이터 타입: <class 'str'>
⚠️ LapNumber 컬럼이 없습니다. (거리 기반 슬라이싱 필요)


7. 결측치 확인

In [11]:
# 확인할 컬렉션 리스트
target_collections = {
    "races_meta": {}, # 전체 조회
    "results": {},    # 전체 조회
    "laps": {},       # 전체 조회
    "telemetry": {"limit": 100000} # ⚠️ 너무 많으므로 최신 10만개만 샘플링
}

for col_name, options in target_collections.items():
    print(f"📊 [ {col_name} ] 결측치 확인")

    # 데이터 로드
    if "limit" in options:
        # 최신순으로 정렬해서 가져오기 (ObjectId 역순)
        cursor = db[col_name].find().sort('_id', -1).limit(options["limit"])
        print(f"   (⚠️ 데이터가 많아 최신 {options['limit']}개만 샘플링하여 검사합니다)")
    else:
        cursor = db[col_name].find()

    df = pd.DataFrame(list(cursor))

    if df.empty:
        print("   ❌ 데이터가 없습니다.\n")
        continue

    # _id 컬럼 제거
    if '_id' in df.columns:
        df = df.drop(columns=['_id'])

    # 결측치 계산
    null_counts = df.isnull().sum()
    null_percent = (df.isnull().sum() / len(df)) * 100

    # 결측치가 하나라도 있는 컬럼만 필터링
    missing_df = pd.DataFrame({'Missing Count': null_counts, 'Percent (%)': null_percent})
    missing_df = missing_df[missing_df['Missing Count'] > 0].sort_values('Percent (%)', ascending=False)

    if not missing_df.empty:
        display(missing_df.style.format({'Percent (%)': '{:.2f}%'}))
    else:
        print("   ✅ 결측치 없음 (Clean Data)")

    print("-" * 50)

📊 [ races_meta ] 결측치 확인


,Missing Count,Percent (%)
TotalLaps,171,50.00%


--------------------------------------------------
📊 [ results ] 결측치 확인


,Missing Count,Percent (%)
Q3,5168,75.59%
TimeDelta,4580,66.99%
Q2,4319,63.17%
Q1,3465,50.68%
Position,6,0.09%


--------------------------------------------------
📊 [ laps ] 결측치 확인


,Missing Count,Percent (%)
Sector1_Sec,3400,1.54%
TyreLife,1510,0.69%
Sector3_Sec,103,0.05%
Sector2_Sec,96,0.04%


--------------------------------------------------
📊 [ telemetry ] 결측치 확인
   (⚠️ 데이터가 많아 최신 100000개만 샘플링하여 검사합니다)
   ✅ 결측치 없음 (Clean Data)
--------------------------------------------------


📚 컬럼(Feature) 상세 설명서
<br/>
1. races_meta (경기 메타데이터)
- Round: 시즌 내 경기 순서 (예: 1, 2, 3...)
- SessionType: Qualifying (예선) 또는 Race (본선)
- Circuit: 서킷 이름 (예: Bahrain Grand Prix)
- Avg_AirTemp: 평균 기온 (°C) - 타이어 예열과 엔진 쿨링에 영향
- Avg_TrackTemp: 평균 트랙 온도 (°C) - 타이어 마모도(Degradation) 분석의 핵심 변수
- IsRainy: 우천 여부 (True/False) - 비가 오면 랩타임 분석에서 제외하거나 별도 취급해야 함
<br/>

2. results (경기 결과)
- Driver: 드라이버 이름 약어 (예: VER, HAM)
- DriverNumber: 드라이버 엔트리 넘버 (예: 1, 44) - 텔레메트리 매핑 시 중요
- Position: 최종 순위
- GridPosition: 출발 그리드 순위 - 순위 상승폭(Overtaking) 계산 시 사용
- Status: 완주 여부 (Finished, Collision, Engine 등) - 신뢰성 분석 시 Finished가 아닌 데이터 활용
- Points: 획득 포인트
<br/>

3. laps (랩 타임 데이터)
- LapNumber: 랩 번호
- LapTime_Sec: 해당 랩을 도는 데 걸린 시간 (초) - 가장 중요한 타겟 변수($Y$)
- Sector1_Sec, 2, 3: 섹터별 구간 기록
- Compound: 타이어 종류 (SOFT, MEDIUM, HARD, INTERMEDIATE, WET)
- TyreLife: 해당 타이어를 끼고 달린 랩 수 (타이어 나이)
- IsAccurate: [중요] 기록이 유효한지 여부.
    - True: 정상적인 어택 랩.
    - False: 피트 인/아웃, 세이프티카 상황 등. (분석 시 반드시 True인 것만 필터링해야 함)
<br/>

4. telemetry (초당 주행 데이터)
- Date: 타임스탬프
- Speed: 차량 속도 (km/h) - 최고 속도 분석용
- RPM: 엔진 회전수
- Gear: 현재 기어 단수 (1~8)
- Throttle: 악셀 페달 전개량 (0~100) - 100이면 풀 악셀
- Brake: 브레이크 페달 여부 (0 또는 100, 혹은 압력값)
- DRS: 가변 리어 윙 작동 여부 (0: 닫힘, 10~14: 열림) - 직선 구간에서 속도 차이 유발
- X, Y, Z: 트랙 내 GPS 좌표 - 트랙 맵 그릴 때 사용
- Distance: 해당 랩 시작점으로부터의 거리 (미터) - 그래프 X축(거리) 기준으로 사용